In [1]:
import requests
import pandas as pd
import pickle
from statistics import mean
import os
import json
import yfinance as yf
import matplotlib.pyplot as plt

In [2]:
def apply_portafolio_theory(tickers_composition,dollar_price,interval,period):
    tickers_statistics={}
    total_invested=sum(x[1]*x[2] for x in  tickers_composition)            
    earnings_df = pd.DataFrame()
    portfolio_earning=0
    for ticker,quantity,current_price in tickers_composition:
        #Getting earnings usgin historicals per ticker 
        hist = yf.Ticker(ticker).history(period=period,interval=interval).dropna()
        hist["earnings"]=(hist.Close-hist.Open)*100/hist.Open
        tickers_statistics[ticker]={ \
                                    "earning":hist.earnings.mean(),\
                                    "risk":hist.earnings.std(),\
                                    "performance":hist.earnings.mean()/hist.earnings.std(),\
                                    "portfolio_percentage":current_price*quantity/total_invested \
                                   }
        earnings_df=pd.concat([earnings_df,hist["earnings"]],axis = 1)
        earnings_df.rename(columns = {'earnings':ticker}, inplace = True)
        #Getting portfolio earning using earnings mean and portfolio percentage per ticker
        portfolio_earning+=tickers_statistics[ticker]['earning']*tickers_statistics[ticker]['portfolio_percentage']

    portfolio_risk=0
    cov_matrix=(earnings_df).cov().values
    
    for x in range(len(earnings_df.cov().values)):
        for y in range(len(earnings_df.cov().values[x])):
            portfolio_risk+=(tickers_composition[x][1]*tickers_composition[x][2]/total_invested)*(tickers_composition[y][1]*tickers_composition[y][2]/total_invested)*cov_matrix[x][y]
    portfolio_risk=portfolio_risk**(1/2)
    portfolio_performance=portfolio_earning/portfolio_risk

#    print(f"Portfolio earnings per {interval}: {portfolio_earning} \
#            \nPortfolio risk per {interval}: {portfolio_risk} \
#            \nPortfolio performance: {portfolio_performance} \
#          ")
    return portfolio_performance,portfolio_earning,portfolio_risk


In [3]:
dollar_price=17.47
tickers_composition= [("AAPL",1), \
                     ("AMZN",7) , \
                     ("GOOGL",20)]
ticker_and_price=[ (ticker[0],ticker[1], yf.Ticker(ticker[0]).history(period="1d",interval="1d").iloc[0]['Close']*dollar_price) for ticker in tickers_composition ]
apply_portafolio_theory(ticker_and_price,dollar_price,"1mo","5y")

(np.float64(0.22455128512599548),
 np.float64(1.7058279005821195),
 np.float64(7.596607160920862))

In [6]:
##S&P500 List
a='''
VLO
VTR
TSLA
'''

In [9]:
results=[]
try_tickers=a.split()
dollar_price=20.16
interval="1wk"
period="2y"
money_to_invest=8500.05
tickers_composition=[("AAPL",1),
                     ("AMZN",7) ,\
                     ("GOOGL",20), \
                     ("LLY",1), \
                     ("META",1), \
                     ("MSFT",1), \
                     ("NEE",1), \
                     ("PG",1), \
                     ("TMUS",1), \
                     ("TSLA",1), \
                     ("V",1), \
                     ("VOO",1), \
                     ("XLK",1)#, \
                     
]
ticker_and_price=[ (ticker[0],ticker[1], yf.Ticker(ticker[0]).history(period="1d",interval="1d").iloc[0]['Close']*dollar_price) for ticker in tickers_composition ]

for ticker in try_tickers:
    try:
        print("Using: "+ticker)
        ticker_candidate_price=yf.Ticker(ticker).history(period="1d",interval="1d").iloc[0]['Close']*dollar_price
        if ticker_candidate_price > money_to_invest:
            print(ticker+" is to expensive. Moving to next one")
            continue
        else:
            ticker_and_price.append((ticker, 1,ticker_candidate_price))
            p_perf,p_earn,p_risk=apply_portafolio_theory(ticker_and_price,dollar_price,interval,period)
            results.append((ticker,p_perf,p_earn,p_risk)) 
            ticker_and_price.pop()
    except :
        print ("Failed with: "+ticker)
        ticker_and_price.pop()
print("-----FINISHED--------")

Using: VLO
Using: VTR
Using: TSLA
-----FINISHED--------


In [26]:
pd.DataFrame(results).sort_values(by=[1],ascending=False).head()

,0,1,2,3
1,VTR,0.176356,0.522565,2.963131
2,VRSN,0.176207,0.517707,2.938053
0,VLO,0.175981,0.518587,2.946826


In [114]:
look_details=('CHKEW', 1, 'NA', 10000)
tickers_composition.append(look_details)
print(apply_portafolio_theory(tickers_composition))
tickers_composition.pop()

(0.15855428460132498, 0.4671408000647136, 2.94625150773636)


('CHKEW', 1, 'NA', 10000)

In [19]:
ticker_and_price

[('AAPL', 1, 3163.9811686706544),
 ('AMZN', 7, 2147.1338634490967),
 ('GOOGL', 20, 2113.5982791137694),
 ('LLY', 1, 7660.317953796386),
 ('META', 1, 4807.91),
 ('MSFT', 1, 5857.266070251465),
 ('NEE', 1, 1293.3448373413084),
 ('PG', 1, 2558.629285125732),
 ('TMUS', 1, 2240.212174682617),
 ('TSLA', 1, 4457.839546203613),
 ('V', 1, 3916.6501626586914),
 ('VOO', 1, 6933.827499999999),
 ('XLK', 1, 2963.2809417724607)]